# LangSmith & simple evaluation

# simple evaluation 

In [4]:
# ========== 1. 패키지 설치 ==========
!pip install langsmith langchain langchain-openai langchain-community chromadb tiktoken --quiet

In [5]:


# ========== 2. 환경 설정 ==========
import os
import getpass
import json
import pandas as pd
from typing import List, Dict
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain.schema import Document
from langchain.prompts import PromptTemplate

In [ ]:

# OpenAI API 키 설정 (사용자가 입력해야 함)
# from google.colab import userdata
# api_key=userdata.get('api_key')
# os.environ["OPENAI_API_KEY"] = api_key

from dotenv import load_dotenv

load_dotenv()
# OpenAI API 클라이언트 생성
OPENAPI_KEY = os.getenv("OPENAI_API_KEY")
LangSmith_KEY = os.getenv("LANGCHAIN_API_KEY")


# 2) LangSmith 연동 필수 환경변수
os.environ["LANGCHAIN_TRACING_V2"] = "true"      # 트레이싱 활성화
os.environ["LANGSMITH_ENDPOINT"]   = "https://api.smith.langchain.com"  # 기본값
os.environ["LANGSMITH_PROJECT"]    = "RAG_EV_ex1"                 # 수업용 프로젝트명

In [7]:


# ========== 3. 샘플 문서 준비 ==========
sample_documents = [
    """
    인공지능(AI)은 인간의 학습능력, 추론능력, 지각능력을 인공적으로 구현한 컴퓨터 시스템입니다.
    머신러닝은 AI의 한 분야로, 데이터를 통해 컴퓨터가 스스로 학습하도록 하는 기술입니다.
    딥러닝은 머신러닝의 한 방법으로, 인공신경망을 여러 층으로 쌓아 복잡한 패턴을 학습합니다.
    최근에는 GPT, BERT 같은 대규모 언어모델이 AI 발전을 주도하고 있습니다.
    """,
    """
    자연어처리(NLP)는 컴퓨터가 인간의 언어를 이해하고 처리하는 기술입니다.
    최근 트랜스포머 모델의 등장으로 NLP 분야는 큰 발전을 이루었습니다.
    BERT는 양방향 인코더 표현을 사용하여 문맥을 이해하는 모델입니다.
    GPT는 자기회귀 방식으로 텍스트를 생성하는 언어모델입니다.
    """,
    """
    RAG(Retrieval-Augmented Generation)는 검색과 생성을 결합한 AI 기술입니다.
    외부 지식베이스에서 관련 정보를 검색한 후, 이를 바탕으로 답변을 생성합니다.
    이를 통해 LLM의 환각(hallucination) 문제를 완화하고 최신 정보를 반영할 수 있습니다.
    벡터 데이터베이스를 활용하여 효율적인 유사도 검색을 수행합니다.
    """
]

# Document 객체로 변환
documents = [Document(page_content=doc, metadata={"source": f"doc_{i}"})
             for i, doc in enumerate(sample_documents)]

# ========== 4. 텍스트 분할 ==========
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=300,
    chunk_overlap=50,
    separators=["\n\n", "\n", ".", " "]
)

chunks = text_splitter.split_documents(documents)
print(f"생성된 청크 수: {len(chunks)}")

# ========== 5. LLM 기반 QA 생성 클래스 ==========
class SyntheticQAGenerator:
    def __init__(self, llm):
        self.llm = llm

    def generate_simple_questions(self, text: str, num_questions: int = 3) -> List[Dict]:
        """단순 사실 확인 질문 생성"""
        prompt = PromptTemplate(
            input_variables=["text", "num_questions"],
            template="""다음 텍스트를 읽고 직접적인 사실 확인 질문과 답변을 생성하세요.

텍스트:
{text}

요구사항:
- {num_questions}개의 질문-답변 쌍을 생성하세요
- 질문은 텍스트에서 직접 찾을 수 있는 내용이어야 합니다
- 답변은 간결하고 정확해야 합니다

다음 JSON 형식으로 출력하세요:
[
    {{
        "question": "질문 내용",
        "answer": "답변 내용"
    }},
    ...
]
"""
        )

        response = self.llm.invoke(prompt.format(text=text, num_questions=num_questions))

        try:
            # JSON 파싱
            qa_pairs = json.loads(response.content)
            return [
                {
                    "question": qa["question"],
                    "ground_truth": qa["answer"],
                    "contexts": [text],
                    "evolution_type": "simple"
                }
                for qa in qa_pairs
            ]
        except:
            # JSON 파싱 실패 시 대체 방법
            return self._parse_fallback(response.content, text, "simple")

    def generate_reasoning_questions(self, texts: List[str], num_questions: int = 2) -> List[Dict]:
        """추론이 필요한 질문 생성"""
        combined_text = "\n\n".join(texts)
        prompt = PromptTemplate(
            input_variables=["text", "num_questions"],
            template="""다음 텍스트들을 읽고 추론이 필요한 질문과 답변을 생성하세요.

텍스트:
{text}

요구사항:
- {num_questions}개의 질문-답변 쌍을 생성하세요
- 질문은 여러 정보를 종합하거나 추론이 필요한 내용이어야 합니다
- 답변은 논리적이고 근거가 명확해야 합니다

다음 JSON 형식으로 출력하세요:
[
    {{
        "question": "추론이 필요한 질문",
        "answer": "논리적인 답변"
    }},
    ...
]
"""
        )

        response = self.llm.invoke(prompt.format(text=combined_text, num_questions=num_questions))

        try:
            qa_pairs = json.loads(response.content)
            return [
                {
                    "question": qa["question"],
                    "ground_truth": qa["answer"],
                    "contexts": texts,
                    "evolution_type": "reasoning"
                }
                for qa in qa_pairs
            ]
        except:
            return self._parse_fallback(response.content, texts, "reasoning")

    def generate_multi_context_questions(self, texts: List[str], num_questions: int = 2) -> List[Dict]:
        """여러 문서를 참조해야 하는 질문 생성"""
        combined_text = "\n\n".join(texts)
        prompt = PromptTemplate(
            input_variables=["text", "num_questions"],
            template="""다음 여러 텍스트를 종합하여 답해야 하는 질문과 답변을 생성하세요.

텍스트:
{text}

요구사항:
- {num_questions}개의 질문-답변 쌍을 생성하세요
- 질문은 여러 문서의 정보를 종합해야 답할 수 있어야 합니다
- 답변은 포괄적이고 상세해야 합니다

다음 JSON 형식으로 출력하세요:
[
    {{
        "question": "종합적인 질문",
        "answer": "포괄적인 답변"
    }},
    ...
]
"""
        )

        response = self.llm.invoke(prompt.format(text=combined_text, num_questions=num_questions))

        try:
            qa_pairs = json.loads(response.content)
            return [
                {
                    "question": qa["question"],
                    "ground_truth": qa["answer"],
                    "contexts": texts,
                    "evolution_type": "multi_context"
                }
                for qa in qa_pairs
            ]
        except:
            return self._parse_fallback(response.content, texts, "multi_context")

    def _parse_fallback(self, content: str, contexts, evolution_type: str) -> List[Dict]:
        """JSON 파싱 실패 시 대체 파싱"""
        qa_pairs = []
        lines = content.strip().split('\n')

        current_q = None
        for line in lines:
            if '"question"' in line.lower() or 'q:' in line.lower():
                # 질문 추출
                if ':' in line:
                    current_q = line.split(':', 1)[1].strip().strip('"').strip(',')
            elif '"answer"' in line.lower() or 'a:' in line.lower():
                # 답변 추출
                if current_q and ':' in line:
                    answer = line.split(':', 1)[1].strip().strip('"').strip(',').strip('}')
                    qa_pairs.append({
                        "question": current_q,
                        "ground_truth": answer,
                        "contexts": contexts if isinstance(contexts, list) else [contexts],
                        "evolution_type": evolution_type
                    })
                    current_q = None

        return qa_pairs

# ========== 6. 테스트 데이터셋 생성 ==========
# LLM 초기화
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.3)

# QA 생성기 초기화
qa_generator = SyntheticQAGenerator(llm)

print("\n===== 합성 테스트 데이터 생성 시작 =====")

# 1. Simple 질문 생성
simple_qa = []
for chunk in chunks[:2]:
    qa_pairs = qa_generator.generate_simple_questions(chunk.page_content, num_questions=3)
    simple_qa.extend(qa_pairs)

print(f"✅ Simple 질문 {len(simple_qa)}개 생성 완료")

# 2. Reasoning 질문 생성
reasoning_qa = qa_generator.generate_reasoning_questions(
    [chunk.page_content for chunk in chunks[:3]],
    num_questions=3
)
print(f"✅ Reasoning 질문 {len(reasoning_qa)}개 생성 완료")

# 3. Multi-context 질문 생성
multi_context_qa = qa_generator.generate_multi_context_questions(
    [chunk.page_content for chunk in chunks],
    num_questions=2
)
print(f"✅ Multi-context 질문 {len(multi_context_qa)}개 생성 완료")

# 모든 QA 결합
all_qa = simple_qa + reasoning_qa + multi_context_qa
test_df = pd.DataFrame(all_qa)

print(f"\n총 {len(test_df)}개의 테스트 케이스 생성 완료!")

# ========== 7. 결과 확인 ==========
print("\n===== 생성된 테스트 데이터셋 미리보기 =====")
for idx, row in test_df.head(3).iterrows():
    print(f"\n[테스트 케이스 {idx + 1}]")
    print(f"질문: {row['question']}")
    print(f"정답: {row['ground_truth']}")
    print(f"유형: {row['evolution_type']}")
    print(f"컨텍스트 수: {len(row['contexts'])}")
    print("-" * 50)

# ========== 8. LangSmith에 데이터셋 업로드 ==========
from langsmith import Client

try:
    client = Client()

    # 데이터셋 생성
    dataset_name = "synthetic_qa_dataset_v3"
    dataset = client.create_dataset(
        dataset_name=dataset_name,
        description="LLM으로 생성한 합성 Q&A 테스트 데이터셋"
    )

    # 각 테스트 케이스를 LangSmith에 추가
    for idx, row in test_df.iterrows():
        client.create_example(
            dataset_id=dataset.id,
            inputs={
                "question": row['question'],
                "contexts": row['contexts']
            },
            outputs={
                "answer": row['ground_truth']
            },
            metadata={
                "evolution_type": row['evolution_type']
            }
        )

    print(f"\n✅ 데이터셋이 LangSmith에 업로드되었습니다!")
    print(f"데이터셋 이름: {dataset_name}")
    print(f"테스트 케이스 수: {len(test_df)}")

except Exception as e:
    print(f"\n⚠️ LangSmith 업로드 실패: {e}")
    print("로컬에만 저장합니다.")

# ========== 9. 데이터셋 통계 및 분석 ==========
print("\n===== 데이터셋 통계 =====")
print(test_df['evolution_type'].value_counts())

# 질문/답변 길이 분석
test_df['question_length'] = test_df['question'].str.len()
test_df['answer_length'] = test_df['ground_truth'].str.len()

print(f"\n질문 길이:")
print(f"  평균: {test_df['question_length'].mean():.1f} 글자")
print(f"  최소: {test_df['question_length'].min()} 글자")
print(f"  최대: {test_df['question_length'].max()} 글자")

print(f"\n답변 길이:")
print(f"  평균: {test_df['answer_length'].mean():.1f} 글자")
print(f"  최소: {test_df['answer_length'].min()} 글자")
print(f"  최대: {test_df['answer_length'].max()} 글자")

# ========== 10. CSV로 저장 ==========
test_df.to_csv('synthetic_qa_dataset.csv', index=False, encoding='utf-8-sig')
print("\n✅ 데이터셋이 'synthetic_qa_dataset.csv'로 저장되었습니다.")

# ========== 11. 품질 검증 ==========
print("\n===== 데이터 품질 검증 =====")

# 중복 질문 체크
duplicate_questions = test_df[test_df.duplicated(['question'], keep=False)]
if len(duplicate_questions) > 0:
    print(f"⚠️ 중복 질문 발견: {len(duplicate_questions)}개")
else:
    print("✅ 중복 질문 없음")

# 빈 값 체크
empty_questions = test_df[test_df['question'].str.strip() == '']
empty_answers = test_df[test_df['ground_truth'].str.strip() == '']

if len(empty_questions) > 0 or len(empty_answers) > 0:
    print(f"⚠️ 빈 값 발견 - 질문: {len(empty_questions)}개, 답변: {len(empty_answers)}개")
else:
    print("✅ 모든 질문과 답변이 정상적으로 생성됨")

# ========== 12. 샘플 평가용 함수 (선택사항) ==========
def evaluate_qa_quality(question: str, answer: str, context: str) -> Dict:
    """간단한 QA 품질 평가"""
    # 답변이 컨텍스트에 기반하는지 체크
    context_words = set(context.lower().split())
    answer_words = set(answer.lower().split())

    # 답변과 컨텍스트의 단어 겹침 비율
    overlap_ratio = len(answer_words & context_words) / len(answer_words) if answer_words else 0

    return {
        "question_length": len(question),
        "answer_length": len(answer),
        "context_overlap": overlap_ratio,
        "is_complete": '?' in question and len(answer) > 10
    }

# 샘플 평가
print("\n===== 샘플 품질 평가 (처음 3개) =====")
for idx, row in test_df.head(3).iterrows():
    quality = evaluate_qa_quality(
        row['question'],
        row['ground_truth'],
        row['contexts'][0] if row['contexts'] else ""
    )
    print(f"\n케이스 {idx+1}:")
    print(f"  컨텍스트 겹침: {quality['context_overlap']:.2%}")
    print(f"  완성도: {'✅' if quality['is_complete'] else '⚠️'}")

print("\n🎉 합성 테스트 데이터셋 생성 완료!")
print(f"총 {len(test_df)}개의 고품질 QA 쌍이 생성되었습니다.")

생성된 청크 수: 3

===== 합성 테스트 데이터 생성 시작 =====
✅ Simple 질문 6개 생성 완료
✅ Reasoning 질문 3개 생성 완료
✅ Multi-context 질문 2개 생성 완료

총 11개의 테스트 케이스 생성 완료!

===== 생성된 테스트 데이터셋 미리보기 =====

[테스트 케이스 1]
질문: 인공지능(AI)은 무엇을 구현한 컴퓨터 시스템인가요?
정답: 인간의 학습능력, 추론능력, 지각능력을 인공적으로 구현한 컴퓨터 시스템입니다.
유형: simple
컨텍스트 수: 1
--------------------------------------------------

[테스트 케이스 2]
질문: 머신러닝은 AI의 어떤 분야인가요?
정답: AI의 한 분야로, 데이터를 통해 컴퓨터가 스스로 학습하도록 하는 기술입니다.
유형: simple
컨텍스트 수: 1
--------------------------------------------------

[테스트 케이스 3]
질문: 딥러닝은 무엇을 통해 복잡한 패턴을 학습하나요?
정답: 인공신경망을 여러 층으로 쌓아 복잡한 패턴을 학습합니다.
유형: simple
컨텍스트 수: 1
--------------------------------------------------

✅ 데이터셋이 LangSmith에 업로드되었습니다!
데이터셋 이름: synthetic_qa_dataset_v3
테스트 케이스 수: 11

===== 데이터셋 통계 =====
evolution_type
simple           6
reasoning        3
multi_context    2
Name: count, dtype: int64

질문 길이:
  평균: 28.8 글자
  최소: 18 글자
  최대: 37 글자

답변 길이:
  평균: 121.7 글자
  최소: 16 글자
  최대: 329 글자

✅ 데이터셋이 'synthetic_qa_dataset.csv'로 저장되었습니다.

===== 데이터 품질 검증 ==